___
## rnn_pytorch
#### Use pytorch's nn.RNN to create predictions similar to those predicted by da_rnn.
___

In [61]:
import pdb
import  numpy as np
import torch 
from torch import nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import logging
import pandas as pd
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
np.set_printoptions(suppress=True)
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)
df_data = None

In [5]:
class SimpleRnn(nn.Module):
    def __init__(self, input_size, hidden_size=32):
        super(SimpleRnn, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size=input_size,hidden_size=hidden_size,batch_first=True,)
        self.out = nn.Linear(hidden_size, 1)

    def forward(self, x, h_state):
        # x (batch, time_step, input_size)
        # h_state (n_layers, batch, hidden_size)
        # r_out (batch, time_step, hidden_size)
        r_out, h_state = self.rnn(x, h_state)

        outs = []    # save all predictions
        for time_step in range(r_out.size(1)):    # calculate output for each time step
            outs.append(self.out(r_out[:, time_step, :]))
        return torch.stack(outs, dim=1), h_state


In [63]:
df_data = None #pd.read_csv('df_sin_cos.csv')

In [76]:
BATCH_SIZE = 1
TIME_STEP = 10      # rnn time step
INPUT_SIZE = 1      # DEFAULT rnn input size.
LR = 0.02           # learning rate
LABEL_COLUMN = 'y'

rnn = SimpleRnn(INPUT_SIZE)
print(rnn)

optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)   # optimize all cnn parameters
loss_func = nn.MSELoss()

h_state = None      # for initial hidden state

plt.figure(1, figsize=(12, 5))
plt.ion()           # continuously plot

# set up defaults
epochs = 5000
st, ed = 0,epochs*np.pi
all_steps = np.linspace(st, ed, TIME_STEP*epochs, dtype=np.float32, endpoint=False)  # float32 for converting torch FloatTensor
# x_train = np.sin(all_steps)
# y_train = np.cos(all_steps)
x_train = np.sin(all_steps).reshape(-1,TIME_STEP,1)
y_train = np.cos(all_steps).reshape(-1,1)


if df_data is not None:
    epochs = len(df_data)
    x_cols = list(filter(lambda c: c != LABEL_COLUMN,df_data.columns.values))
    INPUT_SIZE= len(x_cols)
    x_train = df_data[[x_cols]].as_matrix().reshape(-1,INPUT_SIZE,1)
    y_train = df_data[LABEL_COLUMN].as_matrix().reshape(-1,1)

for step in range(epochs):
#     start, end = step * np.pi, (step+1)*np.pi   # time range
    
    
#     x_np2 = x_train[step*TIME_STEP:(step+1)*TIME_STEP]
#     y_np2 = y_train[step*TIME_STEP:(step+1)*TIME_STEP]    
#     x = torch.from_numpy(x_np2[np.newaxis, :, np.newaxis])    # shape (batch, time_step, input_size)
#     y = torch.from_numpy(y_np2[np.newaxis, :, np.newaxis])

    x_np2 = x_train[step]
    x = torch.from_numpy(x_np2)    # shape (batch, time_step, input_size)
    y = torch.from_numpy(y_np2)
    prediction, h_state = rnn(x, h_state)   # rnn output
    print(prediction.shape,y.shape)
    if step>1:
        break
    # !! next step is important !!
    h_state = h_state.data        # repack the hidden state, break the connection from last iteration

    loss = loss_func(prediction, y)         # calculate loss
    optimizer.zero_grad()                   # clear gradients for this training step
    loss.backward()                         # backpropagation, compute gradients
    optimizer.step()                        # apply gradients

    # plotting
    if step % (epochs//10) ==0:
        plt.plot(steps, y_np2.flatten(), 'r-')
        plt.plot(steps, prediction.data.numpy().flatten(), 'b-')
        plt.draw(); plt.pause(0.05)

plt.ioff()
plt.show()


SimpleRnn(
  (rnn): RNN(1, 32, batch_first=True)
  (out): Linear(in_features=32, out_features=1, bias=True)
)
torch.Size([10, 1, 1]) torch.Size([10])


RuntimeError: input and target shapes do not match: input [10 x 1 x 1], target [10] at /Users/soumith/code/builder/wheel/pytorch-src/aten/src/THNN/generic/MSECriterion.c:12

<Figure size 864x360 with 0 Axes>

In [66]:
x_train.shape

(50000, 1, 1)

In [86]:
dff = pd.DataFrame({'x1':np.linspace(0,19,20),'x2':np.linspace(20,39,20),'y':np.linspace(40,59,20)})
df_x = dff[['x1','x2']]
df_y = dff[['y']]
x = df_x.as_matrix()
y = df_y.as_matrix()
print(x.shape,y.shape)

(20, 2) (20, 1)


In [102]:
xr = x.reshape(-1,5,2)

In [103]:
yr = y.reshape(-1,5,1)

In [109]:
xr[np.random.randint(0,len(xr)-1,2)]

array([[[ 5., 25.],
        [ 6., 26.],
        [ 7., 27.],
        [ 8., 28.],
        [ 9., 29.]],

       [[ 0., 20.],
        [ 1., 21.],
        [ 2., 22.],
        [ 3., 23.],
        [ 4., 24.]]])

In [123]:
r = [0,45,90]
c = np.cos(np.radians(r))
r,c,np.degrees(np.arccos(c))

([0, 45, 90],
 array([1.        , 0.70710678, 0.        ]),
 array([ 0., 45., 90.]))

In [124]:
import os
os.getcwd()

'/Users/bperlman1/Documents/billybyte/pyliverisk/jupyter_notebooks/da_rnn'

In [126]:
dff = pd.read_csv('data/uso_201812.csv')

In [127]:
dff.columns.values

array(['close', 'high', 'low', 'open', 'symbol', 'timestamp',
       'tradingDay', 'volume'], dtype=object)